In [1]:
\from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pydicom

     |████████████████████████████████| 1.9MB 7.5MB/s 


In [5]:
!pip install wandb
import wandb
wandb.login()
wandb.init(project='homework3')

wandb: Currently logged in as: ting-liu (use `wandb login --relogin` to force relogin)


In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import csv
import random
import numpy as np
import pandas as pd
import zipfile
import shutil
import progressbar
import albumentations as A
import cv2 as cv
from pydicom import dcmread
from pydicom.data import get_testdata_files
from PIL import Image
from torchvision import datasets, models, transforms, models, utils
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader



In [4]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-ihj68y1p
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-ihj68y1p
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263906 sha256=e9749ffd3477e7ab5600bc68dccec08d83b2877f8d59efb6477c55af5c3344db
  Stored in directory: /tmp/pip-ephem-wheel-cache-pgzddw_s/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2


In [5]:
SavePath = '/content/drive/MyDrive/Colab Notebooks/homework3'
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/homework3/train.csv 的副本')

#df=df.drop_duplicates(subset='patientId')
df[['x','y','width','height']] = df[['x','y','width','height']].div((1024/224))
df=df.round(2)
df[['x','y']] = df[['x','y']].fillna(1)
df[['width','height']] = df[['width','height']].fillna(150)
#df=df.dropna()
#df.iloc[0,0]
len(df)


24181

In [7]:
df_new=df.drop_duplicates(subset='patientId')

In [8]:
df_new[df_new['Target']==1]

,patientId,class,Target,x,y,width,height
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity,1,57.75,33.25,46.59,82.91
8,00704310-78a8-4b38-8475-49f4573b2dbb,Lung Opacity,1,70.66,126.22,35.00,22.75
14,00aecb01-a116-45a2-956c-08d2fa55433f,Lung Opacity,1,63.00,70.44,20.56,29.53
16,00c0b293-48e7-4e16-ac76-9269ba535a62,Lung Opacity,1,66.94,119.00,36.75,53.38
19,00f08de1-517e-4652-a04f-d1dc9ee48593,Lung Opacity,1,39.59,40.25,45.06,110.69
...,...,...,...,...,...,...,...
24144,d91b519f-9216-4ff9-9e2a-3356bfe4e391,Lung Opacity,1,59.06,115.50,42.88,33.91
24147,d9245b95-1261-4dd6-88a8-2c9a69578b0e,Lung Opacity,1,144.16,70.44,35.66,57.53
24158,d930583e-0c91-4b5f-9a23-0f71f0349586,Lung Opacity,1,32.38,132.56,45.06,27.34
24169,d947215e-c4c5-4907-8d47-0735a0ba4fb9,Lung Opacity,1,156.84,117.47,37.84,50.53


In [ ]:
df_new

,patientId,class,Target,x,y,width,height
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal,0,1.00,1.00,150.00,150.00
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal,0,1.00,1.00,150.00,150.00
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal,0,1.00,1.00,150.00,150.00
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal,0,1.00,1.00,150.00,150.00
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity,1,57.75,33.25,46.59,82.91
...,...,...,...,...,...,...,...
24175,d9528d5b-b058-467c-accb-618c100f3696,Lung Opacity,1,50.75,102.59,42.22,71.97
24177,d955b013-2d49-4afd-8218-40f5e3af8312,Normal,0,1.00,1.00,150.00,150.00
24178,d95be64c-1ef5-4521-833f-fc3818885c45,Normal,0,1.00,1.00,150.00,150.00
24179,d95cc647-caee-42f2-8229-0d22d64d92f6,No Lung Opacity / Not Normal,0,1.00,1.00,150.00,150.00


In [ ]:
df

,patientId,class,Target,x,y,width,height
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal,0,1.00,1.00,150.00,150.00
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal,0,1.00,1.00,150.00,150.00
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal,0,1.00,1.00,150.00,150.00
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal,0,1.00,1.00,150.00,150.00
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity,1,57.75,33.25,46.59,82.91
...,...,...,...,...,...,...,...
24176,d9528d5b-b058-467c-accb-618c100f3696,Lung Opacity,1,137.81,94.94,41.78,72.84
24177,d955b013-2d49-4afd-8218-40f5e3af8312,Normal,0,1.00,1.00,150.00,150.00
24178,d95be64c-1ef5-4521-833f-fc3818885c45,Normal,0,1.00,1.00,150.00,150.00
24179,d95cc647-caee-42f2-8229-0d22d64d92f6,No Lung Opacity / Not Normal,0,1.00,1.00,150.00,150.00


In [7]:

#分割data
imgpath = []
imglabel = []

# Load csv and add path
for idx, row in df_new.iterrows():
  imgpath.append('/content/drive/MyDrive/Colab Notebooks/homework3/resize_images/'+row['patientId']+'.png')
  imglabel.append(row["Target"])

len(imgpath)

21166

In [8]:
train_img = imgpath
train_label = np.array(imglabel)

In [ ]:
#print(len(train_img))
#print(len(val_img))
#val_img

In [9]:
class dataset(Dataset):
    
    def __init__(self, traindata,dataframe, trainlabel, transform):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.images = traindata
        self.df = dataframe
        self.label = trainlabel 
        self.transform = transform
        self.image_ids = dataframe['patientId'].unique()
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        imgpath = self.images[index]
        img=Image.open(imgpath)
        img_array = np.array(img)

        labels = self.label[index]  
            

        image_id = self.image_ids[index]
        
        records = self.df[self.df['patientId'] == image_id]
        boxes = records[['x', 'y', 'width', 'height']].values
        #boxes = self.df[['x','y','width','height']].values
        
        obj_ids = np.array(boxes)
        num_objs = len(obj_ids)

        if int(labels) == 0:
          label = np.zeros((num_objs,), dtype=np.int64)
        else:
          label = np.ones((num_objs,), dtype=np.int64)
        label = torch.as_tensor(label,dtype=torch.int64)

        area = boxes[:, 2] * boxes[:, 3]
        area = torch.as_tensor(area, dtype=torch.float32)

        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        #boxes[:,1],boxes[:,3]=boxes[:,3],boxes[:,1]
        boxes=torch.as_tensor(boxes,dtype=torch.float16)
        
        # there is only one class
        #labels = torch.ones((records.shape[0],), dtype=torch.int64)
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.uint8)

        target = {}
        target["boxes"] = boxes
        target["labels"] = label
        target["image_id"] = torch.tensor([index])
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transform is not None:
          img_array, target = self.transform(img_array, target)
      
        
        return img_array,target




In [10]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 30343, done.
remote: Counting objects: 100% (2885/2885), done.
remote: Compressing objects: 100% (689/689), done.
remote: Total 30343 (delta 2208), reused 2767 (delta 2139), pack-reused 27458
Receiving objects: 100% (30343/30343), 38.22 MiB | 31.41 MiB/s, done.
Resolving deltas: 100% (22823/22823), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be376084 version check against PyTorch's CUDA version


In [11]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


      
def get_object_detection_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True,min_size=200)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [12]:
#!pip install albumentations==0.4.6
import albumentations 
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import albumentations as A


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        #transforms.append(A.Flip(0.5),)
      
        transforms.append(T.RandomHorizontalFlip(0.7))
       
    return T.Compose(transforms)
   

In [ ]:
print(T.Compose)

<class 'transforms.Compose'>


In [13]:

dataset_train = dataset(train_img,df, train_label,get_transform(train=True))
dataset_val = dataset(train_img,df, train_label,get_transform(train=False))

for i in range(10):
  img,target=dataset_train[i]
  print(target)
print(type(target['boxes']))

{'boxes': tensor([[ 73.,   1., 223., 151.]], dtype=torch.float16), 'labels': tensor([0]), 'image_id': tensor([0]), 'area': tensor([22500.]), 'iscrowd': tensor([0], dtype=torch.uint8)}
{'boxes': tensor([[ 73.,   1., 223., 151.]], dtype=torch.float16), 'labels': tensor([0]), 'image_id': tensor([1]), 'area': tensor([22500.]), 'iscrowd': tensor([0], dtype=torch.uint8)}
{'boxes': tensor([[  1.,   1., 151., 151.]], dtype=torch.float16), 'labels': tensor([0]), 'image_id': tensor([2]), 'area': tensor([22500.]), 'iscrowd': tensor([0], dtype=torch.uint8)}
{'boxes': tensor([[ 73.,   1., 223., 151.]], dtype=torch.float16), 'labels': tensor([0]), 'image_id': tensor([3]), 'area': tensor([22500.]), 'iscrowd': tensor([0], dtype=torch.uint8)}
{'boxes': tensor([[119.6875,  33.2500, 166.2500, 116.1875],
        [ 45.0000,  33.2500, 101.0625, 132.3750]], dtype=torch.float16), 'labels': tensor([1, 1]), 'image_id': tensor([4]), 'area': tensor([3862.7769, 5549.0400]), 'iscrowd': tensor([0, 0], dtype=torch.ui

In [19]:
dataset_train[21165]

(tensor([[[0.0078, 0.0078, 0.0078,  ..., 0.0157, 0.0118, 0.0078],
          [0.0078, 0.0078, 0.0078,  ..., 0.0196, 0.0157, 0.0118],
          [0.0078, 0.0078, 0.0078,  ..., 0.0235, 0.0196, 0.0118],
          ...,
          [0.6863, 0.7020, 0.7176,  ..., 0.7294, 0.7216, 0.5098],
          [0.6980, 0.7176, 0.7373,  ..., 0.7373, 0.7373, 0.5216],
          [0.5804, 0.6000, 0.6196,  ..., 0.6039, 0.5961, 0.4235]]]),
 {'area': tensor([22500.]),
  'boxes': tensor([[ 73.,   1., 223., 151.]], dtype=torch.float16),
  'image_id': tensor([21165]),
  'iscrowd': tensor([0], dtype=torch.uint8),
  'labels': tensor([0])})

In [14]:
# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset_train)).tolist()
dataset_train = torch.utils.data.Subset(dataset_train, indices[:-4233])
dataset_val = torch.utils.data.Subset(dataset_val, indices[-4233:])

# define training and validation data loaders
data_loader_train = torch.utils.data.DataLoader(
    dataset_train, batch_size=8, shuffle=True, num_workers=16,
    collate_fn=utils.collate_fn 
    )

data_loader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=8, shuffle=False, num_workers=32,
    collate_fn=utils.collate_fn
    )

  

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_object_detection_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.00005,
                             weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [ ]:
'''
# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_val, device=device)
'''

In [19]:
weight_path = SavePath+'/model_weight2.pth'
model.load_state_dict(torch.load(weight_path))

<All keys matched successfully>

In [ ]:
import time
from tqdm import tqdm


# 或者: from evaluate import evaluate
#from tqdm.notebook import tqdm as tqdm
criterion = nn.CrossEntropyLoss()
valid_loss_min = np.Inf # track change in validation loss
itr = 1
num_epochs = 20
total_train_loss = []
total_valid_loss = []
x_axis=[]
y_axis_train=[]
y_axis_validation=[]
losses_value = 0

for epoch in range(num_epochs):

  start_time = time.time()
  # plot 
  x_axis.append(epoch)
  # train ------------------------------

  model.train()
  train_loss = []
 
  pbar = tqdm(data_loader_train, desc='let\'s train')
  for images, targets in pbar:
   
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
    loss_dict = model(images, targets)
    losses = sum(loss for loss in loss_dict.values())
    losses_value = losses.item()
    train_loss.append(losses_value)

    optimizer.zero_grad()
    losses.backward()
    optimizer.step()

   

    pbar.set_description(f"Epoch: {epoch+1}, Batch: {itr}, Loss: {losses_value}")
    itr += 1

  epoch_train_loss = np.mean(train_loss)
  total_train_loss.append(epoch_train_loss)

  # update the learning rate
  if lr_scheduler is not None:
    lr_scheduler.step()
 

  # valid ------------------------------
 
  with torch.no_grad():
    valid_loss = []

    for images, targets in tqdm(data_loader_val):
      images = list(image.to(device) for image in images)
      targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

      loss_dict = model(images, targets)

      losses = sum(loss for loss in loss_dict.values())
      loss_value = losses.item()
      valid_loss.append(loss_value)

      

  epoch_valid_loss = np.mean(valid_loss)
  total_valid_loss.append(epoch_valid_loss)

  if epoch_valid_loss <= valid_loss_min:
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
      valid_loss_min,
      epoch_valid_loss))
      torch.save(model.state_dict(), SavePath+'/model_weight2.pth')
      valid_loss_min = epoch_valid_loss
  
  #plot loss
  y_axis_train.append(epoch_train_loss)
  y_axis_validation.append(epoch_valid_loss)
  # print ------------------------------
  evaluate (model, data_loader_val,device=device)
  
  print(f"Epoch Completed: {epoch+1}/{num_epochs}, Time: {time.time()-start_time}, "
     f"Train Loss: {epoch_train_loss}, Valid Loss: {epoch_valid_loss}")

#plot
data_1,=plt.plot(x_axis,y_axis_train,'r-.^',label='train')
data_2,=plt.plot(x_axis,y_axis_validation,'g--*',label='validation')
plt.tick_params(axis='both',labelsize=24,color='green')
plt.legend(handles=[data_1,data_2])
  



let's train:   0%|          | 0/2117 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/530 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_chec

Validation loss decreased (inf --> 0.078360).  Saving model ...
creating index...
index created!
Test:  [  0/530]  eta: 0:17:45  model_time: 0.3268 (0.3268)  evaluator_time: 0.0676 (0.0676)  time: 2.0101  data: 1.6029  max mem: 1449
Test:  [100/530]  eta: 0:01:40  model_time: 0.1642 (0.1689)  evaluator_time: 0.0100 (0.0124)  time: 0.2061  data: 0.0293  max mem: 1449
Test:  [200/530]  eta: 0:01:10  model_time: 0.1574 (0.1647)  evaluator_time: 0.0101 (0.0114)  time: 0.1858  data: 0.0158  max mem: 1449
Test:  [300/530]  eta: 0:00:47  model_time: 0.1577 (0.1620)  evaluator_time: 0.0101 (0.0110)  time: 0.1721  data: 0.0018  max mem: 1449
Test:  [400/530]  eta: 0:00:26  model_time: 0.1550 (0.1602)  evaluator_time: 0.0094 (0.0108)  time: 0.1739  data: 0.0058  max mem: 1449
Test:  [500/530]  eta: 0:00:06  model_time: 0.1548 (0.1592)  evaluator_time: 0.0102 (0.0107)  time: 0.2174  data: 0.0494  max mem: 1449
Test:  [529/530]  eta: 0:00:00  model_time: 0.1582 (0.1589)  evaluator_time: 0.0100 (0.

let's train:   0%|          | 0/2117 [00:00<?, ?it/s]

DONE (t=0.74s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.062
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.181
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.024
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.011
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.128
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.082
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.185
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.186
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.056
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.400
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.165
Epoch C

100%|██████████| 530/530 [01:28<00:00,  6.01it/s]


...

Epoch: 16, Batch: 33872, Loss: 0.06970230489969254: 100%|██████████| 2117/2117 [11:53<00:00,  2.97it/s]
100%|██████████| 530/530 [01:29<00:00,  5.94it/s]


creating index...
index created!
Test:  [  0/530]  eta: 0:22:21  model_time: 0.5972 (0.5972)  evaluator_time: 0.1008 (0.1008)  time: 2.5317  data: 1.8294  max mem: 1449
Test:  [100/530]  eta: 0:01:36  model_time: 0.1655 (0.1671)  evaluator_time: 0.0098 (0.0114)  time: 0.1837  data: 0.0063  max mem: 1449
Test:  [200/530]  eta: 0:01:10  model_time: 0.1566 (0.1643)  evaluator_time: 0.0102 (0.0111)  time: 0.2217  data: 0.0505  max mem: 1449
Test:  [300/530]  eta: 0:00:47  model_time: 0.1545 (0.1616)  evaluator_time: 0.0100 (0.0109)  time: 0.2015  data: 0.0327  max mem: 1449
Test:  [400/530]  eta: 0:00:26  model_time: 0.1538 (0.1600)  evaluator_time: 0.0097 (0.0108)  time: 0.1804  data: 0.0132  max mem: 1449
Test:  [500/530]  eta: 0:00:06  model_time: 0.1556 (0.1595)  evaluator_time: 0.0096 (0.0108)  time: 0.1724  data: 0.0061  max mem: 1449
Test:  [529/530]  eta: 0:00:00  model_time: 0.1510 (0.1588)  evaluator_time: 0.0101 (0.0108)  time: 0.1800  data: 0.0222  max mem: 1449
Test: Total tim

let's train:   0%|          | 0/2117 [00:00<?, ?it/s]

DONE (t=0.73s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.062
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.182
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.024
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.012
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.129
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.082
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.185
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.185
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.055
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.400
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.161
Epoch C

Epoch: 17, Batch: 35934, Loss: 0.16191452741622925:  97%|█████████▋| 2062/2117 [11:31<00:17,  3.08it/s]

In [22]:
torch.save(model.state_dict(), SavePath+'/model_weight2.pth')

In [22]:
test_df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/homework3/submission.csv 的副本 的副本')
test_df1['class']='NaN'
test_df=test_df1.drop_duplicates(subset='patientId')
imgpath_test = []

for idx, row in test_df.iterrows():
  imgpath_test.append('/content/drive/MyDrive/Colab Notebooks/homework3/resize_testImage/'+row['patientId']+'.png')

Class={'No Lung Opacity / Not Normal or Normal':0,
      'Lung Opacity':1,}


In [24]:
test_df

,patientId,class,Target,x,y,width,height
0,d9625316-b22c-44db-adbe-d99cfbb9e996,NaN,NaN,NaN,NaN,NaN,NaN
1,d9628d5b-2437-4df9-b66c-d35e30f3a40b,NaN,NaN,NaN,NaN,NaN,NaN
2,d9645d7c-415f-420d-b82a-675062674282,NaN,NaN,NaN,NaN,NaN,NaN
3,d967c678-30f0-48b4-9563-167e746e972d,NaN,NaN,NaN,NaN,NaN,NaN
4,d96a84b3-a770-4270-9089-572f2782967d,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
6038,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,NaN,NaN,NaN,NaN,NaN,NaN
6040,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,NaN,NaN,NaN,NaN,NaN,NaN
6042,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,NaN,NaN
6043,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:

# load an instance segmentation model pre-trained on COCO
net_load = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# get the number of input features for the classifier
in_features = net_load.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
net_load.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net_load.to(device)
    
weight_path = SavePath+'/model_weight2.pth'
net_load.load_state_dict(torch.load(weight_path))

net_load.eval()
net_load.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [26]:
transform = transforms.Compose([
       transforms.ToTensor(),
])

class test_set(Dataset):
    def __init__(self,img_path,transform):
        # --------------------------------------------
        # Initialize paths, transforms, and so on
        # --------------------------------------------
        
        self.images = img_path
        self.transform = transform
        
    def __getitem__(self, index):
        # --------------------------------------------
        # 1. Read from file (using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (torchvision.Transform).
        # 3. Return the data (e.g. image and label)
        # --------------------------------------------
        imgpath = self.images[index]
        img=Image.open(imgpath)
      
        
        
        if self.transform is not None:
         
          img = self.transform(img)
      
        
        return img,index


        
    def __len__(self):
        # --------------------------------------------
        # Indicate the total size of the dataset
        # --------------------------------------------
        return len(self.images)

In [27]:
import torch.nn.functional as F

testset = test_set(imgpath_test,transform)
testloader = DataLoader(testset, batch_size=1, shuffle=False, pin_memory=True, num_workers = 16,  collate_fn=utils.collate_fn)

with torch.no_grad():
    score_threshold = 0.73
    pred = []

    for img, index in tqdm(testloader):
      img = list(image.to(device) for image in img)
      outputs = net_load(img)
      num = index[0]
      
      for idx, output in zip(index, outputs):

        boxes = output['boxes'].data.cpu().numpy()
        scores = output['scores'].data.cpu().numpy()

        bool_value = scores >= score_threshold
        boxes = boxes[bool_value].astype(np.float32)
        scores = scores[bool_value]
      
        if len(scores) > 0:
         pred.append(1)
         test_df.at[num, 'Target'] = 1
         test_df.at[num, 'class'] = 'Lung Opacity'

        else:
         pred.append(0)
         test_df.at[num, 'Target'] = 0
         test_df.at[num, 'class'] = 'No Lung Opacity'


        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

        if pred[num] == 0:
          test_df.at[num, 'x'] = 0.0
          test_df.at[num, 'y'] = 0.0
          test_df.at[num, 'width'] = 0.0
          test_df.at[num, 'height'] = 0.0
        elif pred[num] == 1 and len(boxes)==1:
          test_df.at[num, 'x'] = boxes[0, 0]
          test_df.at[num, 'y'] = boxes[0, 1]
          test_df.at[num, 'width'] = boxes[0, 2]
          test_df.at[num, 'height'] = boxes[0, 3]
        else:
          patientId = test_df.at[num, 'patientId']
          #max_value = max(scores)
          max_idx = np.argmax(scores)
          test_df.at[num, 'x'] = boxes[max_idx, 0]
          test_df.at[num, 'y'] = boxes[max_idx, 1]
          test_df.at[num, 'width'] = boxes[max_idx, 2]
          test_df.at[num, 'height'] = boxes[max_idx, 3]
          scores_2 = scores
          scores_2[np.argmax(scores_2)] = np.min(scores)
          max2_idx = np.argmax(scores_2)
          test_df = test_df.append({'patientId':patientId, 'class':'Lung Opacity', 'Target':1, 'x':boxes[max2_idx, 0] , 'y':boxes[max2_idx, 1], 'width':boxes[max2_idx, 2], 'height':boxes[max2_idx, 3]} , ignore_index=True)
          
    


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [28]:
test_df

,patientId,class,Target,x,y,width,height
0,d9625316-b22c-44db-adbe-d99cfbb9e996,Lung Opacity,1.0,204.867813,15.874852,5.013824,6.879252
1,d9628d5b-2437-4df9-b66c-d35e30f3a40b,No Lung Opacity,0.0,0.000000,0.000000,0.000000,0.000000
2,d9645d7c-415f-420d-b82a-675062674282,Lung Opacity,1.0,192.538895,11.595178,8.701050,10.203073
3,d967c678-30f0-48b4-9563-167e746e972d,Lung Opacity,1.0,185.774841,11.105405,21.205902,16.389465
4,d96a84b3-a770-4270-9089-572f2782967d,No Lung Opacity,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
7441,c14d9ceb-019f-45f6-9299-281b58de57df,Lung Opacity,1.0,16.332800,12.934175,5.509039,6.987973
7442,c18d1138-ba74-4af5-af21-bdd4d2c96bb5,Lung Opacity,1.0,176.430466,6.771493,7.628372,8.649753
7443,c19b8a3b-ab4e-4a73-8e13-ec0a84b6b6c7,Lung Opacity,1.0,126.580307,0.000000,97.419693,62.261410
7444,c1ca4417-83a6-43a7-a9bf-7d9587e7f14f,Lung Opacity,1.0,0.000000,0.000000,205.353683,34.984791


In [29]:
test_df.to_csv(SavePath+ '/result.csv', encoding='utf-8')